# Introduction

In this assignment, we learn "Naive Bayes classifier from scratch". We implement Naive Bayes classifier on a broken seven segement display and classify the observations among different classes/labels using probabilities. The Naive Bayes classifier is based on Bayesian Inference in which we build a clssifier which takes an observation and returns its class. In this assignment, we have two datasets train and test with 5000 and 1000 observations respectively. In which, the seven segments of the display "a through g" are the features and the values of these features are binary values `on`(1) or `off`(0) of the malfunctioned seven segment display. Each observation is labeled as the digit which is meant to be displayed by the seven segment display.

## Goal 

Our goal is to build a classifier using the train dataset which takes an observation as input and infer the label(digit) of that observation. The decision to label the observation is based on the probabilities. A digit which has maximum likelihood for that observation is assigned as label.

In [1]:
# load required libraries
import pandas as pd 
import numpy as np  

feature_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g'] 
train_data = pd.read_csv("seven_digits_train.csv")# load train data
test_data = pd.read_csv("seven_digits_test.csv") # load test data

In [2]:
print("Training data shape {}".format(train_data.shape))
print("Test Data shape {}".format(test_data.shape))

Training data shape (5000, 8)
Test Data shape (1000, 8)


#  Classification via Naive Bayes 

The predicted class for a given observation is computed by:

\begin{equation}
p(y|x) = argmax \,\, p(y)*p(x|y)
\end{equation}

where, $p(y|x)$ is the posterior,
        $p(y)$ is the prior,
        $p(x|y)$ is the likelihood or conditional probabaility. 

In this classfier our first step is to compute the `priors` of each class (digit) from the given data which is computed by : `count of observation for class (0 to 9)/Total number of observations`. 
For example, consider $y = 2$, the prior $P(2)$ = count of observation for class $2$ / Total number of observations

The second step is to compute the `conditional probabilites` for each class for each feature. For instance,  

P(x|y) is P(a=1|digit2) = Count of `on` values in feature a in digit 2 examples in the training data/ Total number of digit 2 examples in the training data. Similarly, we compute for every feature a to g and for every digit 0 to 9.

Finally, for a given test observation we compute the probability of all 10 classes by taking product of all conditional probabilities for that class with its prior. Thus, we compute ten probabilities for one observation and considering maximum likelihood we assign the label of digit to every observation in test data.

To make our computation and classification more robust we use log so that the product of very small probabilities would not turn into zero. In that case, if a very small value is zero then product of all the values would be zero. By taking log we take the sum of the prior and conditional probabilities:

The formula for log form of Naive Bayes is:
\begin{equation}
log p(y|x) = argmax \,\, log(p(y)) + log(p(x|y))
\end{equation}


## Compute class priors p(y)

In [3]:
log_p_y = np.log(train_data['digit'].value_counts() / len(train_data['digit']))
print(log_p_y)

3   -2.205458
2   -2.251892
1   -2.259526
7   -2.280824
9   -2.294617
4   -2.308603
6   -2.333044
0   -2.341326
8   -2.349677
5   -2.416874
Name: digit, dtype: float64


## Estimate likelihood p(x|y)

In [4]:
# count features for each digit
feature_count = train_data.groupby('digit')[feature_names].sum()
feature_count


,a,b,c,d,e,f,g
digit,,,,,,,
0,422,409,423,405,73,414,412
1,73,450,451,90,77,65,83
2,451,456,81,443,451,460,83
3,469,471,468,463,477,74,70
4,57,424,433,66,419,70,434
5,380,65,375,387,390,54,377
6,421,60,422,423,407,417,409
7,444,439,448,82,62,88,60
8,412,397,401,411,417,414,409


In [5]:
# normalise counts to get conditional probabilities
feature_log_prob = np.log(feature_count.div(feature_count.sum(axis=1), 'index'))
feature_log_prob

,a,b,c,d,e,f,g
digit,,,,,,,
0,-1.801976,-1.833266,-1.799609,-1.843094,-3.556522,-1.821115,-1.825958
1,-2.871163,-1.052374,-1.050155,-2.661812,-2.817817,-2.987235,-2.742781
2,-1.682119,-1.671094,-3.399138,-1.700017,-1.682119,-1.662360,-3.374746
3,-1.670238,-1.665983,-1.672373,-1.683114,-1.653324,-3.516776,-3.572346
4,-3.508136,-1.501453,-1.480449,-3.361532,-1.513316,-3.302692,-1.478142
5,-1.674634,-3.440418,-1.687879,-1.656381,-1.648659,-3.625821,-1.682560
6,-1.804739,-3.753027,-1.802367,-1.800000,-1.838559,-1.814286,-1.833657
7,-1.296207,-1.307532,-1.287238,-2.985312,-3.264897,-2.914695,-3.297687
8,-1.937903,-1.974990,-1.964965,-1.940333,-1.925840,-1.933061,-1.945211


## Predict on test data 

In [10]:
def predict(new_x, log_p_y, log_p_x_given_y):
    posterior = [log_p_y[digit] for digit in range(10)] # assign priors probability
    for feature in ['a', 'b', 'c', 'd', 'e', 'f', 'g']:
        if new_x[feature] == 1: # feature is on 
            for digit in range(10):
                posterior[digit]+= feature_log_prob.iloc[digit][feature]
    return np.argmax(posterior)

test_data['nb_predicted'] = test_data.apply(predict, 
                                            log_p_y=log_p_y, 
                                            log_p_x_given_y=feature_log_prob, axis=1)

In [7]:
accuracy = 100*sum(test_data['digit']==test_data['nb_predicted'])/len(test_data)
print("Model accuracy is {} %".format(accuracy))

Model accuracy is 60.4 %


In [8]:
# confusion matrix 
pd.crosstab(test_data['digit'],test_data['nb_predicted'])

nb_predicted,0,1,2,3,4,5,6,7,8,9
digit,,,,,,,,,,
0,58,1,2,0,1,3,19,1,7,5
1,1,49,1,4,20,1,1,10,0,2
2,4,1,65,15,1,0,2,0,14,2
3,2,0,2,75,1,1,2,2,12,14
4,1,0,0,2,55,3,0,4,2,24
5,2,0,0,21,0,74,12,1,0,8
6,2,0,3,2,0,7,73,0,11,4
7,0,14,1,20,4,1,4,53,4,1
8,15,0,1,1,3,0,11,0,45,11


## Comparison with Scikit-Learn toolkit implementation

In [9]:
# Classification Using sklearn library to compare with our implementation

from sklearn.naive_bayes import MultinomialNB
import warnings
warnings.filterwarnings('ignore')

# create model
gnb = MultinomialNB(alpha=0)
features = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
# fit on train set and test on test set 
gnb.fit(train_data[features], train_data['digit'])
predicted_sk_learn_nb = gnb.predict(test_data[features])

# Assert that our model predictions are same as sklearn MultinomialNB
assert((predicted_sk_learn_nb == test_data['nb_predicted']).all())

# Conclusion
We classified the $1000$ test observations into $10$ classes with $60.4\%$ accuracy. The diagonal values of the confusion matrix shows the correct predictions by our classifier and values in other cells are the misclassifications for a given class. To verify the correctness of our implementation, we also used Scikit-Learn MultinomialNB (https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) implementation. Our predicted values exactly matches with the Scikit-Learn implementation.

Following are a few interesting insights on this dataset.

- Digit 3 has the maximum correctly classified labels, with 75 observations which are correctly predicted by model. 
- Since our confusion matrix is sparse, we observe some common patterns in the errors made by the classifier.
- Our classifier performs poorly for digit 4 as it misclassify digit 4 as digit 9, 24 times, which is the maximum misclassification for a single class. 
- Assert statement shows that the predicted output of our naive bayes classfier is consistent with the sklearn naive bayes classifer, ensuring correctness of our implementation.   
